In [ ]:
import time
import random
import requests
import pandas as pd
from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'undetected_chromedriver.v2'

In [28]:
tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA', 'META', 'NFLX', 'NVDA', 'AMD', 'BA', 
           'V', 'SPY', 'SPCE', 'FB', 'TWTR', 'BABA', 'MSTR', 'DIS', 'PYPL', 'SHOP', 'COIN', 
           'SQ', 'INTC', 'CSCO', 'IBM', 'GE', 'WMT', 'T']

In [29]:
SEC_BASE_URL = "https://www.sec.gov"

In [30]:
# List of user agents for rotation
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36",
]

In [31]:
# Configure Selenium WebDriver
options = Options()
options.add_argument("--headless")  # Run in headless mode
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Initialize WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

In [32]:
def get_8k_filings(ticker):
    """Fetches the latest 8-K filings for a given ticker."""
    headers = {"User-Agent": random.choice(USER_AGENTS)}
    search_url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={ticker}&type=8-K&dateb=&owner=exclude&count=10"

    print(f"\n🔍 Fetching 8-K filings for {ticker}...")
    driver.get(search_url)
    time.sleep(random.uniform(2, 5))  # Randomized delay

    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # Debug: Print fetched page content
    print(f"📄 [DEBUG] Page Content for {ticker}:\n{soup.prettify()[:500]}...\n")

    filings = []
    for link in soup.find_all("a", href=True):
        if "Archives/edgar/data" in link["href"] and "index.htm" in link["href"]:
            filing_url = SEC_BASE_URL + link["href"].replace("index.htm", "")
            filings.append(filing_url)

    if not filings:
        print(f"⚠️ No 8-K filings found for {ticker}.")
    return filings[:3]

In [33]:
def get_transcript(filing_url):
    """Fetches the transcript from an 8-K filing."""
    headers = {"User-Agent": random.choice(USER_AGENTS)}

    print(f"📄 Fetching filing page: {filing_url}")
    driver.get(filing_url)
    time.sleep(random.uniform(2, 5))  # Randomized delay

    soup = BeautifulSoup(driver.page_source, "html.parser")

    transcript_url = None
    all_links = [SEC_BASE_URL + link["href"] for link in soup.find_all("a", href=True)]
    
    # Debug: Print all found links
    print(f"🔗 [DEBUG] All Links on Filing Page:\n{all_links}\n")

    for link in all_links:
        if "ex99" in link.lower():
            transcript_url = link
            break

    if not transcript_url:
        print(f"⚠️ No transcript link found in {filing_url}")
        return None

    print(f"📝 Fetching transcript from: {transcript_url}")
    driver.get(transcript_url)
    time.sleep(random.uniform(2, 5))  # Randomized delay

    soup = BeautifulSoup(driver.page_source, "html.parser")
    paragraphs = soup.find_all("p")

    # Debug: Print first 500 characters of extracted text
    transcript_text = "\n".join([p.get_text() for p in paragraphs])
    print(f"📜 [DEBUG] Transcript Preview:\n{transcript_text[:500]}...\n")

    return transcript_text if transcript_text else None


In [34]:
# Scrape data
data = []
for ticker in tickers:
    filings = get_8k_filings(ticker)
    
    for filing in filings:
        transcript = get_transcript(filing)
        
        if transcript:
            data.append({"Ticker": ticker, "Transcript": transcript})
        
        sleep_time = random.uniform(5, 10)
        print(f"⏳ Sleeping for {sleep_time:.2f} seconds...\n")
        time.sleep(sleep_time)


🔍 Fetching 8-K filings for AAPL...
📄 [DEBUG] Page Content for AAPL:
<html xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   SEC.gov | Request Rate Threshold Exceeded
  </title>
  <style>
   html {height: 100%}
body {height: 100%; margin:0; padding:0;}
#header {background-color:#003968; color:#fff; padding:15px 20px 10px 20px;font-family:Arial, Helvetica, sans-serif; font-size:20px; border-bottom:solid 5px #000;}
#footer {background-color:#003968; color:#fff; padding:15px 20px;font-fa...

⚠️ No 8-K filings found for AAPL.

🔍 Fetching 8-K filings for MSFT...
📄 [DEBUG] Page Content for MSFT:
<html xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   SEC.gov | Request Rate Threshold Exceeded
  </title>
  <style>
   html {height: 100%}
body {height: 100%; margin:0; padding:0;}
#header {background-color:#003968; color:#fff; padding:15px

KeyboardInterrupt: 

In [23]:
# Close the Selenium driver
driver.quit()

In [26]:
data

[]

In [24]:
df = pd.DataFrame(data)

In [25]:
df

""
